In [11]:
import openai
import numpy as np
import os
from dotenv import load_dotenv
import os
from collections import deque
from typing import Dict, List, Optional, Any
from geopy.geocoders import Nominatim

from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
# Langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain ,LLMCheckerChain
from langchain.callbacks import wandb_tracing_enabled
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    StringPromptTemplate
)
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from typing import Optional
from langchain.chains import SimpleSequentialChain ,SequentialChain
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent,BaseMultiActionAgent
from langchain.agents import AgentType, initialize_agent,AgentExecutor,BaseSingleActionAgent
from langchain.tools import tool
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

from utils.Webscraper import Webscraper
webscraper = Webscraper()

In [12]:
# Load variables from the .env file
load_dotenv('./.env')

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT=os.getenv("LANGCHAIN_ENDPOINT")

# Set the environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [15]:
import yfinance as yf
from datetime import datetime, timedelta

def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}

def get_stock_performance(ticker:str, days: int) -> Dict[str, float]:
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

print(get_current_stock_price("MSFT"))
print(get_stock_performance("MSFT", 30))

{'price': 333.54998779296875, 'currency': 'USD'}
{'percent_change': 1.2560090196670617}


In [6]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool


class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")


class StockPercentChangeInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="Timedelta days to get past date from current date")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to check performance of the stock.
        You should enter the stock ticker symbol recognized by the yahoo finance.
        You should enter days as number of days from today from which performance needs to be check.
        output will be the change in the stock price represented as a percentage.
        """
    args_schema: Type[BaseModel] = StockPercentChangeInput

    def _run(self, ticker: str, days: int):
        response = get_stock_performance(ticker, days)
        return response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [7]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)

tools = [CurrentStockPriceTool(), StockPerformanceTool()]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [8]:
agent.run("What is the current price of Microsoft stock? How it has performed over past 6 months?")



> Entering new AgentExecutor chain...
inputs def call: {'input': 'What is the current price of Microsoft stock? How it has performed over past 6 months?'}
def _take_next_step inputs: {'input': 'What is the current price of Microsoft stock? How it has performed over past 6 months?'}

Invoking: `get_current_stock_price` with `{'ticker': 'MSFT'}`


{'price': 333.54998779296875, 'currency': 'USD'}def _take_next_step inputs: {'input': 'What is the current price of Microsoft stock? How it has performed over past 6 months?'}

Invoking: `get_stock_performance` with `{'ticker': 'MSFT', 'days': 180}`


{'percent_change': 34.754778040690155}def _take_next_step inputs: {'input': 'What is the current price of Microsoft stock? How it has performed over past 6 months?'}
_return: AgentFinish(return_values={'output': 'The current price of Microsoft stock is $333.55 USD. \n\nOver the past 6 months, Microsoft stock has performed well with a 34.75% increase in its price.'}, log='The current price of Mic

'The current price of Microsoft stock is $333.55 USD. \n\nOver the past 6 months, Microsoft stock has performed well with a 34.75% increase in its price.'

In [5]:
model_name = "gpt-3.5-turbo-0613"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature)
llm.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
 'kwargs': {'model_name': 'gpt-3.5-turbo-0613',
  'temperature': 0.0,
  'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}}}

## Test Geopands

# Set up agent for location extraction

In [6]:
def get_coordinates(location:str) -> tuple[float, float]:
    """Returns the latitude and longitude of a location."""
    geolocator = Nominatim(user_agent="geoapi_explorer")
    try:
        location_info = geolocator.geocode(location)
    except:
        return "Location Input must be more general, E.g Country,City,state,street"
    
    if location_info:
        latitude = location_info.latitude
        longitude = location_info.longitude
        return (latitude, longitude)
    else:
        return "Location Input must be more general, E.g Country,City,state,street"
    
# use the function
coordinates = get_coordinates("Bali, Indonesia")
coordinates

(-8.3304977, 115.0906401)

In [7]:
# Set up the base template
template = """Role: You're a geographical expert aimed to identity locations about disruption event from News article. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: coordinates of the location in the format (latitude, longitude)

Article: {input}

Question: List your thoughts and extract location where this disruption event occured, using this get 4 coordinates points ( Latitude logitude ) of area where u think this event occured
{agent_scratchpad}"""

In [8]:
@tool("get_coordinates" ,return_direct=True)
def get_coordinates(location:str) -> tuple[float, float]:
    """Returns the latitude and longitude of a location."""
    geolocator = Nominatim(user_agent="geoapi_explorer")
    try:
        location_info = geolocator.geocode(location)
    except:
        return "Location Input must be more general, E.g Country,City,state,street"
    
    if location_info:
        latitude = location_info.latitude
        longitude = location_info.longitude
        return (latitude, longitude)
    else:
        return "Location Input must be more general, E.g Country,City,state,street"
    

tools = [
    get_coordinates,
]
tool_names = [tool.name for tool in tools]

TASK_LIST = ["Determin Location of Disruption Event of that disaster type","Get coordinates of Location"]

In [9]:
test_article = '''A map of the magnitude 7.1 earthquake that struck the Bali Sea region of Indonesia on Aug 29, 2023. (Photo: United States Geological Survey)

29 Aug 2023 05:00AM
(Updated: 29 Aug 2023 08:26AM)
A strong earthquake of 7.0 magnitude struck deep in the sea north of Bali and Lombok islands in Indonesia early on Tuesday (Aug 29), the European-Mediterranean Seismological Centre (EMSC) said, sending residents running out of buildings.

The quake's epicentre was 203km north of Mataram, Indonesia, and very deep at 516 km below the Earth's surface, EMSC said.

Indonesian and US geological agencies pegged the magnitude at 7.1, with no threat of a tsunami.

The quake was felt just before 4am across coastal areas in Bali and Lombok and was followed by two quakes of magnitude 6.1 and 6.5, according to the Indonesian geological agency.

Many residents and tourists rushed out of their homes and hotels toward higher ground after reporting powerful shockwaves, but the situation returned to normal after they received text messages saying the quake had no potential to trigger a tsunami.

“I thought the walls were going to come down on the hotel,” an Australian tourist said on social media.

Guests at Bali's Mercure Kuta Bali ran out of their rooms after feeling the tremor for a few seconds, hotel manager Suadi told Reuters by phone.

"Several guests left their rooms but were still in the hotel area," he said, adding they have since returned and there was no damage to the building.

People in neighbouring provinces of East Java, Central Java, West Nusa Tenggara and East Nusa Tenggara provinces also felt the tremors and panicked as houses and buildings swayed for several seconds.

There were no immediate reports of damage, Indonesian disaster agency BNPB said.

"The quake is deep so it should not be destructive," BNPB spokesperson Abdul Muhari said.'''

In [10]:
article2 = webscraper.scrapeSingleUrl("https://www.straitstimes.com/asia/east-asia/japan-hopes-to-win-the-world-s-trust-over-fukushima-with-hard-science-objective-facts")
article_text_title = article2.title+article2.text

article_text_title

'Japan hopes to win the world’s trust over Fukushima with hard science, objective factsFUKUSHIMA – Japan wants to win the world’s trust that it is doing the right thing – neither wilfully poisoning the Pacific Ocean nor trying to pull the wool over people’s eyes regarding the safety of its seafood with its treated nuclear wastewater release.  On Sunday, The Straits Times was among the first media outlets – domestic and foreign – to visit the crippled Fukushima Daiichi nuclear plant since the discharge began three days earlier, in a process that will end with its full decommissioning only in 2051.  As at 5pm on Sunday (4pm Singapore time), 76 hours after the gates were opened, operator Tokyo Electric Power Co (Tepco) had released 1,420 tonnes of water that had been treated to remove radioactive materials except tritium.  Also on Sunday, Japan said tests of seawater off the coast did not detect any radioactivity. A day earlier, inspections of fish samples from waters near the plant also 

# Making my own class fuck everything already

In [11]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# response_schemas = [
#     ResponseSchema(name="answer", description="answer to the user's question"),
#     ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
# ]
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
location_extractor_schema = {
    "name": "location_extractor",
    "description": "Extracts location(s) from disruption event",
    "type": "object",
    "properties": {
          "location": {
            "type": "string",
            "description": "location of disruption event, only should include the main location(s) where the disruption event occured"
          }
        },
        "required": ["location"]
      }
    


locaiton_prompt = PromptTemplate(
    template="""Role: You're a geographical expert aimed to identity the location(s) about a singular disruption event from News article.\n\nNews Article:{article}\n\nTask:Determin Location of Disruption Event. Location should include any addresses, cities, countries, and landmarks, output an address searchable in googleMaps.Feedback:{feedback}""",
    input_variables=["article","feedback"],
)
location_extractor_chain = create_structured_output_chain(output_schema=location_extractor_schema,llm = llm,prompt=locaiton_prompt)

In [149]:
output = location_extractor_chain.run(article=article_text_title,feedback="")
output

{'location': 'Fukushima Daiichi nuclear plant'}

In [114]:
location_ouput_schema = {
    "name": "Format_coordinate_Extraction",
    "description": "Formats information about disruption event coordinates to return proper output",
    "type": "object",
  "properties": {
    "list_of_locations": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          },
          "latitude": {
            "type": "number"
          },
          "longitude": {
            "type": "number"
          }
        },
        "required": ["location"]
      }
    }
  },
  "required": ["list_of_locations"]
}

location_output_prompt = PromptTemplate(
    template="""Task: Format the output of the agent to the following schema:\n\nAgent Output: {agent_output}""",
    input_variables=["agent_output"]
)
# This should be the last step in the chain
final_output_chain = create_structured_output_chain(output_schema=location_ouput_schema,llm = llm,prompt=location_output_prompt)


In [63]:
output = final_output_chain.run(agent_output="The coordinates of the locations mentioned in the article are: Fukushima Daiichi nuclear plant: (37.423013499999996, 141.0329324479071), China: (35.000074, 104.999927), Hong Kong: (22.350627, 114.1849161)")
output

In [12]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish


class LocationAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="get_coordinates", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="get_coordinates", tool_input=kwargs["input"], log="")
    
agent = LocationAgent()
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
output = agent_executor.run(input="Fukushima Daiichi nuclear plant")



> Entering new AgentExecutor chain...
(37.423013499999996, 141.0329324479071)


> Finished chain.


In [171]:
agent_executor

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, agent=LocationAgent(), tools=[StructuredTool(name='get_coordinates', description='get_coordinates(location: str) -> tuple[float, float] - Returns the latitude and longitude of a location.', args_schema=<class 'pydantic.v1.main.get_coordinatesSchemaSchema'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<function get_coordinates at 0x0000025010C4ED40>, coroutine=None)], return_intermediate_steps=False, max_iterations=15, max_execution_time=None, early_stopping_method='force', handle_parsing_errors=False, trim_intermediate_steps=-1)

In [169]:
# import logging library
import logging
class LocationCoordinatesExtractor:
    """
    The `LocationCoordinatesExtractor` class is responsible for extracting location coordinates from an article.
    It uses a chain of location extractors and an agent executor to perform this task.
    The class allows for a maximum number of tries to get the correct output before raising an exception.

    Example usage:
    """
    def __init__(self, location_extractor_chain: Chain, agent_executor: AgentExecutor, max_tries: int = 5):
        self.location_extractor_chain: Chain = location_extractor_chain
        self.agent_executor: AgentExecutor = agent_executor
        self.max_tries: int = 2

    
    def run(self, article: str) -> Tuple[float, float]:
        """
        Main method to extract location coordinates from the article.
        It runs the location extractor chain and the agent executor in a loop, with a maximum number of tries.
        If the coordinates output is a tuple, it is returned as the result.
        Otherwise, an exception is raised.
        """

        count = 0
        feedback = ""
        while count < self.max_tries:
            try:
                location = self.location_extractor_chain.run(article=article, feedback=feedback)
                print('location: ', location)
                # log output
                coordinates_output = self.agent_executor.run(input=location["location"])
                print('coordinates_output: ', coordinates_output)

            except Exception as e:
                print('error: ', e)
                feedback = str(e)
                count += 1
            else:
                if isinstance(coordinates_output, tuple):
                    return coordinates_output

        raise Exception("Unable to extract location coordinates from the article")
        # Extract location from article    

In [173]:
locationCoordinatesExtractor = LocationCoordinatesExtractor(location_extractor_chain,agent_executor)
locationCoordinatesExtractor.run(article="NIL")

location:  {'location': 'Unknown'}


> Entering new AgentExecutor chain...
(24.9591486, 95.8030352)


> Finished chain.
coordinates_output:  (24.9591486, 95.8030352)


(24.9591486, 95.8030352)

# Custom Agent and modifying

In [17]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser,Agent
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain import OpenAI, SerpAPIWrapper

from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    AsyncCallbackManagerForToolRun,
    CallbackManagerForChainRun,
    CallbackManagerForToolRun,
    Callbacks,
)

In [20]:
template = """Role: You're a geographical expert aimed to identity the location(s) about a singular disruption event from News article. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: coordinates of the location in the format (latitude, longitude)

Article: {input}

Task: {task}
{agent_scratchpad}"""

TASK_LIST = ["Determin Location of Disruption Event. Location should include any addresses, cities, countries, and landmarks, output an address searchable in googleMaps.","Get coordinates of Location"]

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)


prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    task = TASK_LIST[0],
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps","task"]
)

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:

        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return final_output_chain.run(agent_output = llm_output.split("Final Answer:")[-1].strip())
            # return AgentFinish(
            #     # Return values is generally always a dictionary with a single `output` key
            #     # It is not recommended to try anything else at the moment :)
            #     return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
            #     log=llm_output,
            # )

        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)



output_parser = CustomOutputParser()
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

class LLMTaskActionAgent(BaseSingleActionAgent):
    """Base class for single action agents."""
    
    task_list: List[str]
    """List of tasks the agent has to perform in sequential order."""

    llm_chain: LLMChain
    """LLMChain to use for agent."""
    output_parser: AgentOutputParser
    """Output parser to use for agent."""
    stop: List[str]
    """List of strings to stop on."""

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.

        Returns:
            List of input keys.
        """
        return list(set(self.llm_chain.input_keys) - {"intermediate_steps"})

    def dict(self, **kwargs: Any) -> Dict:
        """Return dictionary representation of agent."""
        _dict = super().dict()
        del _dict["output_parser"]
        return _dict

    def plan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with the observations.
            callbacks: Callbacks to run.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        output = self.llm_chain.run(
            intermediate_steps=intermediate_steps,
            stop=self.stop,
            callbacks=callbacks,
            **kwargs,
        )
        return self.output_parser.parse(output)

    async def aplan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            callbacks: Callbacks to run.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        output = await self.llm_chain.arun(
            intermediate_steps=intermediate_steps,
            stop=self.stop,
            callbacks=callbacks,
            **kwargs,
        )
        return self.output_parser.parse(output)

    def tool_run_logging_kwargs(self) -> Dict:
        return {
            "llm_prefix": "",
            "observation_prefix": "" if len(self.stop) == 0 else self.stop[0],
        }
    
agent = LLMTaskActionAgent(
    task_list=TASK_LIST,
    llm_chain=location_extractor_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [21]:
agent_executor.run(article=article_text_title)

ValueError: Missing some input keys: {'feedback'}